In [188]:
from selectolax.parser import HTMLParser
import re

test1 = "/Users/sharjeelmustafa/Desktop/0000088053-06-000310.txt"
test2 = "/Users/sharjeelmustafa/Desktop/0000088053-17-000170.txt"
test3 = "/Users/sharjeelmustafa/Desktop/0000855887-16-000012.txt"

In [189]:
# REGEX
rm_graphic = r"<DOCUMENT>\n?<TYPE>GRAPHIC(.*\n)*?<\/DOCUMENT>"
rm_table = r"<table(.|\n)*?</table>"
rm_extranl = r"\n{3,}"
rm_empty = r"^\s*$"
rm_spaces = r" {2,}"

In [190]:
text = open(test1, 'r').read()

In [191]:
print("Preprocess: len=" + str(len(text)))
# Remove GRAPHIC
text = re.sub(rm_graphic, "", text)
print("Postprocess: len=" + str(len(text)))

# Remove Tables
text = re.sub(rm_table, "", text, flags=re.IGNORECASE)
print("Postprocess: len=" + str(len(text)))

Preprocess: len=967410
Postprocess: len=956366
Postprocess: len=218565


In [192]:
print(text)

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 HJAyyYUWBqKtPhoXfrivq+n8NckQ3lN3hLwy+UiyQC1arMi4cvppNfIX9zM21fEo
 wBdH32mw/ujChbU1586x+A==

<SEC-DOCUMENT>0000088053-06-000310.txt : 20060310
<SEC-HEADER>0000088053-06-000310.hdr.sgml : 20060310
<ACCEPTANCE-DATETIME>20060310163841
ACCESSION NUMBER:		0000088053-06-000310
CONFORMED SUBMISSION TYPE:	N-CSR
PUBLIC DOCUMENT COUNT:		6
CONFORMED PERIOD OF REPORT:	20051231
FILED AS OF DATE:		20060310
DATE AS OF CHANGE:		20060310
EFFECTIVENESS DATE:		20060310

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			NEW GERMANY FUND INC
		CENTRAL INDEX KEY:			0000858706
		IRS NUMBER:				133555471
		STATE OF INCORPORATION:			MD
		FISCAL YEAR END:			1231

	FILING VALUES:
		FORM TYPE:		N-CSR
		SEC ACT:		1940 Act
		SEC FILE NUMB

In [193]:
html = HTMLParser(text).root.traverse()
count = 0
output = ""
for node in html:
    
    if node.tag.lower() != "p": continue
    
    # Remove same paragraph breaks
    temp = node.text().replace("\n", " ")
    # Use unicode, e.g., \xa0 is the Unicode character for &nbsp;
    temp = temp.replace("\xa0", "\n")
    temp = temp.replace("\u2003", "  ")
    if len(temp) > 2: temp = temp.replace("\n", " ")

    # Remove extra newlines
    print(node.tag, node.attributes)
    print(node.text())
    output += temp.strip() + "\n"
    count+=1

p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
UNITED STATES
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
SECURITIES AND EXCHANGE COMMISSION
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
WASHINGTON, D. C. 20549
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:left;'}
 
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
FORM N-CSR
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:left;'}
 
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:left;'}
 
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:left;'}
 
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
THE NEW GERMANY FUND, INC.
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
(Exact Name of Registrant as Specified in Charter)
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:left;'}
 
p {'style': ' margin-bottom:0pt; margin-top:0pt;text-align:center;'}
343 Park A

In [194]:
output = re.sub(rm_extranl, "\n\n", output)
output = re.sub(rm_empty, "", output, flags=re.MULTILINE)
output = re.sub(rm_spaces, " ", output)
open("output.txt", "w").write(output)

print(count)
# 4159 lines including nbsp
# 2014 lines excluding nbsp

976
